# Continuous SIMP example

## Commented Program

What follows is a program spliced with comments.

In [1]:
using TopOpt

### Define the problem

In [2]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force

problems = Any[
    PointLoadCantilever(Val{:Linear}, (60, 20, 20), (1.0, 1.0, 1.0), E, v, f),
    PointLoadCantilever(Val{:Linear}, (160, 40), (1.0, 1.0), E, v, f),
    HalfMBB(Val{:Linear}, (60, 20), (1.0, 1.0), E, v, f),
    LBeam(Val{:Linear}, Float64; force=f),
    TieBeam(Val{:Quadratic}, Float64),
]
problem_names = [
    "3d cantilever beam", "cantilever beam", "half MBB beam", "L-beam", "tie-beam"
]

i = 2
println(problem_names[i])
problem = problems[i]

cantilever beam


### Parameter settings

In [3]:
V = 0.5 # volume fraction
xmin = 0.001 # minimum density
rmin = 3.0

convcriteria = Nonconvex.KKTCriteria()
x0 = fill(V, TopOpt.getncells(problem))
penalty = TopOpt.PowerPenalty(1.0)
solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)
comp = Compliance(solver)
filter = if problem isa TopOptProblems.TieBeam
    identity
else
    DensityFilter(solver; rmin=rmin)
end
obj = x -> comp(filter(PseudoDensities(x)))

#1 (generic function with 1 method)

Define volume constraint

In [4]:
volfrac = Volume(solver)
constr = x -> volfrac(filter(PseudoDensities(x))) - V
model = Model(obj)
addvar!(model, zeros(length(x0)), ones(length(x0)))
add_ineq_constraint!(model, constr)
alg = MMA87()

nsteps = 8
ps = range(1.0, 5.0; length=nsteps + 1)

1.0:0.5:5.0

exponentially decaying tolerance from 10^-2 to 10^-4

In [5]:
tols = exp10.(range(-2, -4; length=nsteps + 1))
x = x0
for j in 1:(nsteps + 1)
    p = ps[j]
    tol = tols[j]
    TopOpt.setpenalty!(solver, p)
    options = MMAOptions(; tol=Tolerance(; kkt=tol), maxiter=1000)
    res = optimize(model, alg, x; options, convcriteria)
    global x = res.minimizer
end

@show obj(x)
@show constr(x)

Internal error: encountered unexpected error in runtime:
TypeError(func=:<:, context="", expected=Type, got=TApprox<:NonconvexMMA.AbstractMMAApprox)
jl_type_error_rt at /buildworker/worker/package_linux64/build/src/rtutils.c:119
jl_type_error at /buildworker/worker/package_linux64/build/src/rtutils.c:127
jl_f_issubtype at /buildworker/worker/package_linux64/build/src/builtins.c:491
apply_type_nothrow at ./compiler/tfuncs.jl:1245
_builtin_nothrow at ./compiler/tfuncs.jl:1566
stmt_effect_free at ./compiler/ssair/queries.jl:36
check_effect_free! at ./compiler/ssair/inlining.jl:1128 [inlined]
process_simple! at ./compiler/ssair/inlining.jl:1185
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1358
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:80
jfptr_ssa_inlining_passNOT._15146.clone_1 at /opt/hostedtoolcache/julia/1.7.3/x64/lib/julia/sys.so (unknown line)
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2247 [inlined]
jl_apply_generic at /buildworker/worker/pack

-7.8393375124719e-10

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(problem; topology = x, default_exagg_scale = 0.07,
    scale_range = 10.0, vector_linewidth = 3, vector_arrowsize = 0.5,
)
Makie.display(fig)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*